In [1]:
import os
import numpy as np
from matplotlib import pyplot
from scipy import optimize
import sys
import pandas as pd

# Cargando el dataset

In [2]:
data1 = pd.read_csv('DATASETRAW1.csv',delimiter=',')  
data1

,Marca,"Modelo (Ej: 2021, 2016)",Número de puertas,Número de asientos,Condición del vehículo,Tipo,Transmisión,Tipo de Combustible,Kilometraje (recorrido en Km),Color,Procedencia,Cilindrada (en cc),Estado de la pintura,¿La pintura está en buen estado?,¿El vehículo tuvo algún accidente?,¿El vehículo cuenta con todos los accesorios?,Costo en $us
0,12,2017,5,5,3,2,1,1,42397.0,4,4,1200,1,1,2,2,27000.0
1,12,2013,4,5,2,2,1,1,60.0,14,3,3500,1,1,2,2,21500.0
2,19,2019,5,12,2,4,1,1,0.0,7,3,3700,1,1,2,2,37800.0
3,30,2020,2,3,2,1,1,3,850.0,14,4,800,1,1,2,2,3900.0
4,10,2001,5,5,3,2,1,1,24900.0,10,4,3000,1,1,2,2,10500.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
930,12,1997,3,4,3,3,2,2,183000.0,7,2,4200,1,1,2,1,13800.0
931,26,2009,5,4,3,2,1,1,200000.0,14,2,2400,2,1,2,1,13000.0
932,17,1992,5,4,3,1,2,1,300.0,7,3,1600,2,1,2,2,5000.0
933,13,2012,5,4,2,2,1,1,90000.0,7,2,2967,2,1,2,2,24900.0


In [3]:
data1.describe()

,Marca,"Modelo (Ej: 2021, 2016)",Número de puertas,Número de asientos,Condición del vehículo,Tipo,Transmisión,Tipo de Combustible,Kilometraje (recorrido en Km),Color,Procedencia,Cilindrada (en cc),Estado de la pintura,¿La pintura está en buen estado?,¿El vehículo tuvo algún accidente?,¿El vehículo cuenta con todos los accesorios?,Costo en $us
count,935.000000,935.000000,935.000000,935.000000,935.000000,935.000000,935.000000,935.000000,9.350000e+02,935.000000,935.000000,935.000000,935.000000,935.000000,935.000000,935.000000,935.000000
mean,15.831016,2011.817112,4.305882,4.632086,2.474866,1.669519,1.408556,1.027807,7.522827e+04,7.190374,3.351872,2456.873797,1.116578,1.075936,1.889840,1.232086,21503.034225
std,10.964479,8.151912,0.888011,0.964819,0.714757,0.777688,0.491830,0.182995,4.523031e+05,4.703422,1.069949,2114.253096,0.321088,0.265037,0.313257,0.422389,13379.477433
min,1.000000,1895.000000,2.000000,2.000000,1.000000,1.000000,1.000000,1.000000,0.000000e+00,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,12.000000
25%,8.000000,2010.000000,4.000000,4.000000,2.000000,1.000000,1.000000,1.000000,1.430000e+04,3.000000,3.000000,1600.000000,1.000000,1.000000,2.000000,1.000000,13200.000000
50%,12.000000,2013.000000,5.000000,5.000000,3.000000,1.000000,1.000000,1.000000,4.500000e+04,7.000000,3.000000,2000.000000,1.000000,1.000000,2.000000,1.000000,18500.000000
75%,26.000000,2016.000000,5.000000,5.000000,3.000000,2.000000,2.000000,1.000000,8.350000e+04,12.000000,4.000000,3000.000000,1.000000,1.000000,2.000000,1.000000,26000.000000
max,40.000000,2022.000000,5.000000,12.000000,4.000000,4.000000,2.000000,3.000000,1.364597e+07,14.000000,8.000000,40000.000000,2.000000,2.000000,2.000000,2.000000,99900.000000


In [4]:
data = data1.to_numpy()

**Filtrado de datos falsos**

In [5]:
def filtrar(matriz):
  c=0
  for i in matriz[:,11]:
    if i < 700 or i >8000:
      matriz=np.delete(matriz,c,axis=0)
    c=c+1

  k=0
  for i in matriz[:,16]:
    if i <3000:
      matriz=np.delete(matriz,k,axis=0)
    k=k+1 

  f=0
  for i in matriz[:,1]:
    if i <1970:
      matriz=np.delete(matriz,f,axis=0)
    f=f+1
  return matriz

In [6]:
data_fil=filtrar(data)
data_fil.shape

(912, 17)

**Asignando X, generando Y**

In [7]:
def generarY(x):
  y1= []
  for i in x[:,16]:
    if i <= 5000:
      y1.append(0)

    if i <=10000 and i>5000:
      y1.append(1)

    if i <=15000 and i>10000:
      y1.append(2)

    if i <=25000 and i>15000:
      y1.append(3)

    if i >25000:
      y1.append(4)

    y=np.array(y1)
  return y


In [9]:
x=data_fil[:900]   #datos para entrenar
xtest= data_fil[900:]  #datos para testear
y=generarY(x)

In [10]:
def  featureNormalize(x):
    x_norm = x.copy()
    mu = np.zeros(x.shape[1])
    sigma = np.zeros(x.shape[1])

    mu = np.mean(x, axis = 0)
    sigma = np.std(x, axis = 0)
    x_norm = (x - mu) / sigma
    
    return x_norm, mu, sigma

In [11]:
x_norm,X_mean,X_std=featureNormalize(x)

# Funciones de activación y pérdida

In [12]:
def softmax(x):
    return np.exp(x) / np.exp(x).sum(axis=-1,keepdims=True)

In [13]:
# aplica softmax + cross entropy de manera estable

def crossentropy(output, target):
    logits = output[np.arange(len(output)),target]
    entropy = - logits + np.log(np.sum(np.exp(output),axis=-1))
    return entropy

def grad_crossentropy(output, target):
    answers = np.zeros_like(output)
    answers[np.arange(len(output)),target] = 1    
    return (- answers + softmax(output)) / output.shape[0]

# Implementación

In [14]:
class Perceptron():
  def __init__(self, inputs, outputs):
    self.w = np.random.normal(loc=0.0, 
          scale = np.sqrt(2/(inputs+outputs)), 
          size = (inputs, outputs)) 
    self.ws = []
    self.loss = crossentropy
    self.grad_loss = grad_crossentropy
    
  def __call__(self, w, x):
    return np.dot(x, w) 

  def fit(self, x, y, epochs, lr, verbose=True, log_each=1):
    x = np.c_[np.ones(len(x)), x]
    for epoch in range(1,epochs+1):
        # Batch Gradient Descent
        y_hat = self(self.w, x)  
        # función de pérdida
        l = self.loss(y_hat, y).mean()
        # derivadas
        dldh = self.grad_loss(y_hat, y)
        dhdw = x
        dldw = np.dot(dhdw.T, dldh)
        # actualizar pesos
        self.w = self.w - lr*dldw
        # guardar pesos para animación
        self.ws.append(self.w.copy())
        # print loss
        if verbose and not epoch % log_each:
            print(f"Epoch {epoch}/{epochs} Loss {l}")
            
  def predict(self, x):
    x = np.c_[np.ones(len(x)), x]
    return self(self.w, x)

In [15]:
m,n=x.shape
perceptron = Perceptron(n+1,5)
epochs, lr = 100, 4
perceptron.fit(x_norm, y, epochs, lr, log_each=10)

Epoch 10/100 Loss 0.6994880979843102
Epoch 20/100 Loss 0.6069361316848019
Epoch 30/100 Loss 0.548482869758741
Epoch 40/100 Loss 0.5097837962137732
Epoch 50/100 Loss 0.48173561836094425
Epoch 60/100 Loss 0.4599313121861933
Epoch 70/100 Loss 0.4421250867205232
Epoch 80/100 Loss 0.42706150002056426
Epoch 90/100 Loss 0.4140006317858889
Epoch 100/100 Loss 0.40247981753280604


In [16]:
def evaluate(x):
    x = (x - X_mean) / X_std 
    y = perceptron.predict(x)
    y_probas = softmax(y)
    return np.argmax(y_probas, axis=1)

In [17]:
def clases(y):
  clase=[]
  for i in range(y.size):
    if y[i] == 0:
      clase.append('Barato')
    if y[i] == 1:
      clase.append('Muy accesible')
    if y[i] == 2:
      clase.append('Accesible')
    if y[i] == 3:
      clase.append('Algo caro')
    if y[i] == 4:
      clase.append('Caro')
  return np.array(clase)

Con el data set de cotización de movilidades, construir un modelo neuronal que permita clasificar a identificar el tipo de costo de una movilidad, de acuerdo al siguiente detalle:



*   Caro (de 25000 $us para arriba).

*   Algo caro (entre 15000 $us - 25000 $us).

*   Accesible (entre 10000 $us - 15000 $us).
*   Muy accesible (entre 5000 $us - 10000 $us).

*   Barato (por debajo de 5000 $us)

In [18]:
ypredict=evaluate(xtest)
ypredict=clases(ypredict)
testeo=np.column_stack((xtest,ypredict))
testeo=pd.DataFrame(testeo,columns=['Marca','Modelo','Puertas','Asientos','Estado del auto','Tipo','Transmicion','Combustible','Kilometraje','Color','Procedencia','Pilindrada','Estado pintura','Condicion2','Accidentes?','Accesorios','Precio en $us','Clase'])
testeo

,Marca,Modelo,Puertas,Asientos,Estado del auto,Tipo,Transmicion,Combustible,Kilometraje,Color,Procedencia,Pilindrada,Estado pintura,Condicion2,Accidentes?,Accesorios,Precio en $us,Clase
0,2.0,2015.0,5.0,4.0,2.0,1.0,1.0,1.0,100884.0,14.0,3.0,2487.0,1.0,1.0,2.0,1.0,22600.0,Algo caro
1,10.0,2014.0,5.0,4.0,2.0,2.0,1.0,1.0,101500.0,14.0,4.0,2400.0,2.0,1.0,2.0,1.0,20490.0,Algo caro
2,19.0,2005.0,4.0,4.0,3.0,1.0,2.0,1.0,108000.0,14.0,2.0,1600.0,2.0,1.0,2.0,2.0,7000.0,Muy accesible
3,2.0,2020.0,5.0,4.0,2.0,2.0,2.0,1.0,108500.0,1.0,6.0,1496.0,2.0,1.0,2.0,1.0,19500.0,Algo caro
4,19.0,2015.0,4.0,4.0,2.0,3.0,2.0,1.0,120000.0,14.0,4.0,2500.0,2.0,1.0,2.0,1.0,21500.0,Algo caro
5,4.0,2016.0,4.0,4.0,2.0,1.0,2.0,1.0,123456.0,14.0,4.0,1600.0,1.0,1.0,2.0,1.0,8000.0,Muy accesible
6,19.0,2000.0,5.0,4.0,3.0,1.0,1.0,1.0,132727.0,14.0,4.0,5000.0,2.0,1.0,2.0,2.0,7500.0,Muy accesible
7,12.0,1997.0,3.0,4.0,3.0,3.0,2.0,2.0,183000.0,7.0,2.0,4200.0,1.0,1.0,2.0,1.0,13800.0,Accesible
8,26.0,2009.0,5.0,4.0,3.0,2.0,1.0,1.0,200000.0,14.0,2.0,2400.0,2.0,1.0,2.0,1.0,13000.0,Accesible
9,17.0,1992.0,5.0,4.0,3.0,1.0,2.0,1.0,300.0,7.0,3.0,1600.0,2.0,1.0,2.0,2.0,5000.0,Muy accesible
